# Ukázka výpočtu vzpěru
jedná se o krtáký program pro kontrolu výpočtu vzpěru v Pythonu

## Podpůrné funkce
abychom mohli vypočíst samotný vzpěr, nejprve musíme si zavést podpůrné funkce, které pak budeme volat v hlavním programu

### Kritická síla
nejprve si zavedeme funkci pro výpočet kritické síly

In [1]:
# ========
# skript pro vypočet kontroly vzperu dle zadaní
# vstupy: J_min, S, l, typ vzperu, F, E, lambda_m
# vystup: F_kr, lambda_vyp, k, posouzení vzperu podle Eulera
# ========

import numpy as np
def vzper(E,Jmin,S,L,typ_vzperu,F,lambda_m):
    # určení typu vzperu
    match typ_vzperu:
        case 1:
            typ = 2 # jednostraně vetknuty
        case 2:
            typ = 1 # jednostraně kloubová podpora, druhý konec posuvná vazba
        case 3:
            typ = 1/np.sqrt(2) # jednostraně vetknuty, druhý konec kloubová podpora
        case 4:
            typ = 1/2 # oboustranně vetknuty
        case _:
            print("Neznámý typ vzpěru, zadjete číslo 1-4.")

    # vypočet kriticke sily

    l0 = L*typ
    Fkr = (np.pi**2*E*Jmin)/l0**2
    k = Fkr/F # bezpecnost
    # výpočet štíhlosti vzperu
    i = np.sqrt(Jmin/S) # polomer kvadratickeho momentu
    lambda_vyp = l0/i #vypoctena stihlost
        
    if lambda_vyp >= lambda_m:
        print("Vzper je bezpečný dle kriteria štíhlosti.")
    else:
        print("Vzper není bezpečný dle kriteria štíhlosti.")

    return Fkr, lambda_vyp, k, typ





### Štíhlost
jako další si zavedeme funkci pro výpočet štíhlosti $\lambda$

In [3]:
# ========
# vypocet stihlosti
# vstupy: L0, S, Jmin
# vystup: lambda_vyp 
# ========
import numpy as np

def stihlost(L0, S, Jmin):
    """Vypočet štíhlosti vzpěry.

    Args:
        l (float): délka vzpěry [mm]
        i_g (float): poloměr setrvačnosti průřezu [mm]

    Returns:
        float: štíhlost vzpěry
    """
    i = np.sqrt(Jmin / S)
    lambda_vyp = L0 / i
    return lambda_vyp

### Výpočet délky
jelikož v některých příkladech, chceme počítat délku prutu, je zde i funkce na tento výpočet
délku vypočítává z definice kritické síly a typ vzpěru

In [5]:
# ==========
# fce pro výpočet délky vzpěry z bezpecnosti a zatěžující síly
# vstupy: J_min, S, typ vzperu, F, E, lambda_m, k
# výstup: L
# ==========
import numpy as np
def vzper_delka(E,Jmin,typ_vzperu,F,k):
    # určení typu vzperu
    match typ_vzperu:
        case 1:
            typ = 2 # jednostraně vetknuty
        case 2:
            typ = 1 # jednostraně kloubová podpora, druhý konec posuvná vazba
        case 3:
            typ = 1/np.sqrt(2) # jednostraně vetknuty, druhý konec kloubová podpora
        case 4:
            typ = 1/2 # oboustranně vetknuty
        case _:
            print("Neznámý typ vzpěru, zadjete číslo 1-4.")

    # výpočet délky vzpěry
    L0 = np.sqrt((np.pi**2*E*Jmin)/(F*k))
    L = L0/typ
    return L

In [7]:
# ==========
# definice proměnných
# ==========
J_min = 5.67e4  # minimální kvadratický moment průřezu [mm4]
S = 800     # plocha průřezu [mm2]
L = 2000        # délka vzperu [mm]
typ_vzperu = 3      # typ vzperu (1 pro jednostraně vetknutý, 2 pro oboustranně vetknutý atd.)
F = 35000    # působící síla [N]
E = 2.1e5     # modul pružnosti materiálu [MPa]
lambda_m = 108.0  # mezní štíhlost
# ==========

# výpočet vzperu
F_kr, lambda_vyp, k1, typ = vzper(E, J_min, S, L, typ_vzperu, F, lambda_m)
print(f"Kritická síla: {F_kr:.2f} N")
print(f"Vypočtená štíhlost: {lambda_vyp:.2f}")
print(f"Bezpečnost: {k1:.2f}")

# vypočet délky vzpěry pro zadanou bezpečnost a sílu
k2 = k1+1/2*k1
F2 = 45000  # nová zatěžující síla [N]
l2 = vzper_delka(E, J_min, typ_vzperu, F2, k2)

print(f"Délka vzpěry pro bezpečnost {k2:.2f} je {l2:.2f} mm ")

# vypocet stihlosti s novou delkou
l02 = l2*typ
lambda_vyp2 = stihlost(l02, S, J_min)
print(f"Štíhlost vzpěry s délkou {l2:.2f} mm je {lambda_vyp2:.2f} ")


Vzper je bezpečný dle kriteria štíhlosti.
Kritická síla: 58758.69 N
Vypočtená štíhlost: 167.98
Bezpečnost: 1.68
Délka vzpěry pro bezpečnost 2.52 je 1440.16 mm 
Štíhlost vzpěry s délkou 1440.16 mm je 120.96 
